https://github.com/geifmany/cifar-vgg/blob/master/cifar10vgg.py

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
print(f"Tensorflow {tf.__version__}")

2025-10-20 17:17:53.679553: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-20 17:17:53.695669: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-20 17:17:54.271968: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-20 17:17:56.597273: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

Tensorflow 2.20.0


In [2]:
# Function to load the CIFAR-10 dataset
import pickle
from keras.utils import to_categorical
def unpickle(file):
    """load the cifar-10 data"""

    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data


def load_cifar_10_data(data_dir, negatives=False):
    """
    Return train_data, train_filenames, train_labels, test_data, test_filenames, test_labels
    """

    # get the meta_data_dict
    # num_cases_per_batch: 1000
    # label_names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    # num_vis: :3072

    meta_data_dict = unpickle(data_dir + "/batches.meta")
    cifar_label_names = meta_data_dict[b'label_names']
    cifar_label_names = np.array(cifar_label_names)

    # training data
    cifar_train_data = None
    cifar_train_filenames = []
    cifar_train_labels = []

    for i in range(1, 6):
        cifar_train_data_dict = unpickle(data_dir + "/data_batch_{}".format(i))
        if i == 1:
            cifar_train_data = cifar_train_data_dict[b'data']
        else:
            cifar_train_data = np.vstack((cifar_train_data, cifar_train_data_dict[b'data']))
        cifar_train_filenames += cifar_train_data_dict[b'filenames']
        cifar_train_labels += cifar_train_data_dict[b'labels']

    cifar_train_data = cifar_train_data.reshape((len(cifar_train_data), 3, 32, 32))
    if negatives:
        cifar_train_data = cifar_train_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        cifar_train_data = np.rollaxis(cifar_train_data, 1, 4)
    cifar_train_filenames = np.array(cifar_train_filenames)
    cifar_train_labels = np.array(cifar_train_labels)

    cifar_test_data_dict = unpickle(data_dir + "/test_batch")
    cifar_test_data = cifar_test_data_dict[b'data']
    cifar_test_filenames = cifar_test_data_dict[b'filenames']
    cifar_test_labels = cifar_test_data_dict[b'labels']

    cifar_test_data = cifar_test_data.reshape((len(cifar_test_data), 3, 32, 32))
    if negatives:
        cifar_test_data = cifar_test_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        cifar_test_data = np.rollaxis(cifar_test_data, 1, 4)
    cifar_test_filenames = np.array(cifar_test_filenames)
    cifar_test_labels = np.array(cifar_test_labels)

    return cifar_train_data, cifar_train_filenames, to_categorical(cifar_train_labels), \
        cifar_test_data, cifar_test_filenames, to_categorical(cifar_test_labels), cifar_label_names



In [3]:
import ANNarchy
from ANNarchy.extensions.ann_to_snn_conversion import ANNtoSNNConverter
ANNarchy.clear()
snn_converter = ANNtoSNNConverter(
    input_encoding='IB', 
    hidden_neuron='IaF',
    read_out='spike_count',
)


ANNarchy 4.8 (4.8.2.5) on linux (posix).


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import regularizers
from keras.models import Model

In [5]:
num_classes = 10
weight_decay = 0.0005
x_shape = (32, 32, 3)

inputs = Input(shape=x_shape)

# --- Block 1 ---
x = Conv2D(64, (3, 3), padding='same',
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d')(inputs)
x = Activation('relu', name='activation')(x)
x = BatchNormalization(name='batch_normalization')(x)
x = Dropout(0.3, name='dropout')(x)

x = Conv2D(64, (3, 3), padding='same',
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d_1')(x)
x = Activation('relu', name='activation_1')(x)
x = BatchNormalization(name='batch_normalization_1')(x)
x = MaxPooling2D(pool_size=(2, 2), name='max_pooling2d')(x)

# --- Block 2 ---
x = Conv2D(128, (3, 3), padding='same', 
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d_2')(x)
x = Activation('relu', name='activation_2')(x)
x = BatchNormalization(name='batch_normalization_2')(x)
x = Dropout(0.4, name='dropout_1')(x)

x = Conv2D(128, (3, 3), padding='same',
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d_3')(x)
x = Activation('relu', name='activation_3')(x)
x = BatchNormalization(name='batch_normalization_3')(x)
x = MaxPooling2D(pool_size=(2, 2), name='max_pooling2d_1')(x)

# --- Block 3 ---
x = Conv2D(256, (3, 3), padding='same', 
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d_4')(x)
x = Activation('relu', name='activation_4')(x)
x = BatchNormalization(name='batch_normalization_4')(x)
x = Dropout(0.4, name='dropout_2')(x)

x = Conv2D(256, (3, 3), padding='same', 
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d_5')(x)
x = Activation('relu', name='activation_5')(x)
x = BatchNormalization(name='batch_normalization_5')(x)
x = Dropout(0.4, name='dropout_3')(x)

x = Conv2D(256, (3, 3), padding='same',
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d_6')(x)
x = Activation('relu', name='activation_6')(x)
x = BatchNormalization(name='batch_normalization_6')(x)
x = MaxPooling2D(pool_size=(2, 2), name='max_pooling2d_2')(x)

# --- Block 4 ---
x = Conv2D(512, (3, 3), padding='same',
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d_7')(x)
x = Activation('relu', name='activation_7')(x)
x = BatchNormalization(name='batch_normalization_7')(x)
x = Dropout(0.4, name='dropout_4')(x)

x = Conv2D(512, (3, 3), padding='same',
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d_8')(x)
x = Activation('relu', name='activation_8')(x)
x = BatchNormalization(name='batch_normalization_8')(x)
x = Dropout(0.4, name='dropout_5')(x)

x = Conv2D(512, (3, 3), padding='same',
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d_9')(x)
x = Activation('relu', name='activation_9')(x)
x = BatchNormalization(name='batch_normalization_9')(x)
x = MaxPooling2D(pool_size=(2, 2), name='max_pooling2d_3')(x)

# --- Block 5 ---
x = Conv2D(512, (3, 3), padding='same',
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d_10')(x)
x = Activation('relu', name='activation_10')(x)
x = BatchNormalization(name='batch_normalization_10')(x)
x = Dropout(0.4, name='dropout_6')(x)

x = Conv2D(512, (3, 3), padding='same',
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d_11')(x)
x = Activation('relu', name='activation_11')(x)
x = BatchNormalization(name='batch_normalization_11')(x)
x = Dropout(0.4, name='dropout_7')(x)

x = Conv2D(512, (3, 3), padding='same',
           kernel_regularizer=regularizers.l2(weight_decay), name='conv2d_12')(x)
x = Activation('relu', name='activation_12')(x)
x = BatchNormalization(name='batch_normalization_12')(x)
x = MaxPooling2D(pool_size=(2, 2), name='max_pooling2d_4')(x)
x = Dropout(0.5, name='dropout_8')(x)

# --- Dense Layers ---
x = Flatten(name='flatten')(x)
x = Dense(512, kernel_regularizer=regularizers.l2(weight_decay), name='dense')(x)
x = Activation('relu', name='activation_13')(x)
x = BatchNormalization(name='batch_normalization_13')(x)
x = Dropout(0.5, name='dropout_9')(x)

outputs = Dense(num_classes,activation='softmax', name='dense_1')(x)


model = Model(inputs=inputs, outputs=outputs, name='cifar10_vgg_functional')


2025-10-20 17:18:05.807214: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
# Load the pretrained Keras model
model.load_weights('cifar10vgg.h5')

#model.summary()
model.save('cifar10vgg.keras')
model.summary()



Model: "cifar10_vgg_functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 15,001,418 (57.23 MB)

 Trainable params: 14,991,946 (57.19 MB)

 Non-trainable params: 9,472 (37.00 KB)

In [7]:
################## Bias test ##################
import tensorflow as tf

# Load your trained model with biases
orig_model = tf.keras.models.load_model("cifar10vgg.keras")

from keras.layers import Input, Conv2D, Activation, BatchNormalization, Dropout, MaxPooling2D, Flatten, Dense
from keras import regularizers, Model

def create_bias_free_model():
    weight_decay = 0.0005
    x_shape = (32, 32, 3)
    inputs = Input(shape=x_shape)

    x = Conv2D(64, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(weight_decay))(inputs)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Conv2D(64, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Conv2D(128, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    x = Conv2D(128, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Conv2D(256, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    x = Conv2D(256, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    x = Conv2D(256, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Conv2D(512, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    x = Conv2D(512, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    x = Conv2D(512, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Flatten()(x)
    x = Dense(512, use_bias=False, kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    outputs = Dense(10, use_bias=False, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

bias_free_model = create_bias_free_model()

import numpy as np
import tensorflow as tf

for orig_layer, new_layer in zip(orig_model.layers, bias_free_model.layers):
    orig_weights = orig_layer.get_weights()
    new_weights = new_layer.get_weights()

    # Skip layers with no weights
    if len(orig_weights) == 0 or len(new_weights) == 0:
        continue

    # If both have same number of weights (e.g., BatchNorm), copy directly
    if len(orig_weights) == len(new_weights):
        new_layer.set_weights(orig_weights)
    # If original has bias but new one doesn’t, drop bias
    elif len(orig_weights) == 2 and len(new_weights) == 1:
        new_layer.set_weights([orig_weights[0]])
    else:
        print(f"Skipping layer {orig_layer.name}: weight shape mismatch "
              f"{[w.shape for w in orig_weights]} vs {[w.shape for w in new_weights]}")



In [9]:
net = snn_converter.load_keras_model("cifar10vgg.keras", show_info=True)


* Input layer: input_layer, (32, 32, 3)
* InputLayer skipped.
* Conv2D layer: conv2d, (32, 32, 64) 
* Activation skipped.
* BatchNormalization skipped.
* Dropout skipped.
* Conv2D layer: conv2d_1, (32, 32, 64) 
* Activation skipped.
* BatchNormalization skipped.
* MaxPooling2D layer: max_pooling2d, (16, 16, 64) 
* Conv2D layer: conv2d_2, (16, 16, 128) 
* Activation skipped.
* BatchNormalization skipped.
* Dropout skipped.
* Conv2D layer: conv2d_3, (16, 16, 128) 
* Activation skipped.
* BatchNormalization skipped.
* MaxPooling2D layer: max_pooling2d_1, (8, 8, 128) 
* Conv2D layer: conv2d_4, (8, 8, 256) 
* Activation skipped.
* BatchNormalization skipped.
* Dropout skipped.
* Conv2D layer: conv2d_5, (8, 8, 256) 
* Activation skipped.
* BatchNormalization skipped.
* Dropout skipped.
* Conv2D layer: conv2d_6, (8, 8, 256) 
* Activation skipped.
* BatchNormalization skipped.
* MaxPooling2D layer: max_pooling2d_2, (4, 4, 256) 
* Conv2D layer: conv2d_7, (4, 4, 512) 
* Activation skipped.
* Bat

In [8]:
#train_data, train_filenames, train_labels, test_data, test_filenames, test_labels, label_names = load_cifar_10_data('cifar-10-batches-py')
# Download data
#test_data = test_data.astype('float32') / 255
#train_data = train_data.astype('float32') / 255

# Load and recompile the model to fix metrics issue
from keras.datasets import cifar10
import keras
model = keras.models.load_model('cifar10vgg.keras')
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train, axis=(0, 1, 2, 3))
X_train = (x_train-mean)/(std+1e-7)
X_test = (x_test-mean)/(std+1e-7)

In [9]:
# Step 1: Start from pretrained model
bias_free_model = create_bias_free_model()  # your functional bias-free copy
# Load all weights except biases (like before)
# ...

# Step 2: Compile and fine-tune for a few epochs
bias_free_model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

bias_free_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=5,  # a few epochs is often enough
)


Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 574s 364ms/step - accuracy: 0.3228 - loss: 3.6204 - val_accuracy: 0.3166 - val_loss: 3.6591
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 620s 397ms/step - accuracy: 0.4467 - loss: 3.1131 - val_accuracy: 0.4173 - val_loss: 3.3852
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 633s 405ms/step - accuracy: 0.5180 - loss: 2.8760 - val_accuracy: 0.4877 - val_loss: 3.0390
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 633s 405ms/step - accuracy: 0.5754 - loss: 2.6895 - val_accuracy: 0.5972 - val_loss: 2.6484
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 632s 404ms/step - accuracy: 0.6186 - loss: 2.5363 - val_accuracy: 0.6026 - val_loss: 2.5942


In [ ]:
test_metrics = bias_free_model.evaluate(x=X_test, y=y_test, batch_size=50, verbose=1, return_dict=True)
print("Accuracy keras: ", test_metrics['accuracy'])

200/200 ━━━━━━━━━━━━━━━━━━━━ 35s 176ms/step - accuracy: 0.5522 - loss: 2.8768
Accuracy keras:  0.5522000193595886


In [10]:
bias_free_model.save("cifar10vgg_bias_free.keras")

In [11]:
net = snn_converter.load_keras_model("cifar10vgg_bias_free.keras", show_info=True)


* Input layer: input_layer_2, (32, 32, 3)
* InputLayer skipped.
* Conv2D layer: conv2d_10, (32, 32, 64) 
* Activation skipped.
* BatchNormalization skipped.
* Dropout skipped.
* Conv2D layer: conv2d_11, (32, 32, 64) 
* Activation skipped.
* BatchNormalization skipped.
* MaxPooling2D layer: max_pooling2d_4, (16, 16, 64) 
* Conv2D layer: conv2d_12, (16, 16, 128) 
* Activation skipped.
* BatchNormalization skipped.
* Dropout skipped.
* Conv2D layer: conv2d_13, (16, 16, 128) 
* Activation skipped.
* BatchNormalization skipped.
* MaxPooling2D layer: max_pooling2d_5, (8, 8, 128) 
* Conv2D layer: conv2d_14, (8, 8, 256) 
* Activation skipped.
* BatchNormalization skipped.
* Dropout skipped.
* Conv2D layer: conv2d_15, (8, 8, 256) 
* Activation skipped.
* BatchNormalization skipped.
* Dropout skipped.
* Conv2D layer: conv2d_16, (8, 8, 256) 
* Activation skipped.
* BatchNormalization skipped.
* MaxPooling2D layer: max_pooling2d_6, (4, 4, 256) 
* Conv2D layer: conv2d_17, (4, 4, 512) 
* Activation 

In [16]:
predictions_snn = snn_converter.predict(X_test[:50], duration_per_sample=100)

100%|██████████| 50/50 [20:09<00:00, 24.18s/it]


In [17]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
#(x_train, y_train), (x_test, y_test) = cifar10.load_data()
#y_train = keras.utils.to_categorical(y_train, 10)
#y_test = keras.utils.to_categorical(y_test, 10)
# Convert one-hot labels (returned by load_cifar_10_data) to integer class indices
y_true = y_test[:50]
if y_true.ndim == 2 and y_true.shape[1] > 1:
    y_true = np.argmax(y_true, axis=1)

print(classification_report(y_true[:50], predictions_snn))
print("Test accuracy of the SNN:", accuracy_score(y_true[:50], predictions_snn))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         9
           7       0.00      0.00      0.00         4
           8       0.08      1.00      0.15         4
           9       0.00      0.00      0.00         8

    accuracy                           0.08        50
   macro avg       0.01      0.10      0.01        50
weighted avg       0.01      0.08      0.01        50

Test accuracy of the SNN: 0.08


/home/sist/projects/test_ann_to_snn/venv_annarchy/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/sist/projects/test_ann_to_snn/venv_annarchy/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/sist/projects/test_ann_to_snn/venv_annarchy/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavio

In [13]:
# Load and recompile the model to fix metrics issue
from keras.datasets import cifar10
import keras
model = keras.models.load_model('cifar10vgg.keras')
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train, axis=(0, 1, 2, 3))
X_train = (x_train-mean)/(std+1e-7)
X_test = (x_test-mean)/(std+1e-7)


In [14]:
# Recompile the model to fix any potential metrics compilation issues
model.compile(
    optimizer='sgd',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
test_metrics = model.evaluate(x=X_test, y=y_test, batch_size=50, verbose=1, return_dict=True)
print("Accuracy keras: ", test_metrics['accuracy'])

200/200 ━━━━━━━━━━━━━━━━━━━━ 42s 205ms/step - accuracy: 0.9359 - loss: 0.4467
Accuracy keras:  0.9358999729156494
